In [1]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from datasets import load_dataset
dataset = load_dataset("beomi/koAlpaca-v1.1a")
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [10]:
print(dataset['train']['instruction'][0])
print(dataset['train']['output'][0])
print(dataset['train']['url'][0])

양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.
https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268


In [11]:
# openai 파인 튜닝
# SFT(Supervised Fine-Tuning) 포멧
# json
{
    'messages': [
        {'role': 'user', 'content': '지금 몇 시야?'},
        {'role': 'assistant', 'content': '지금은 오후 2시입니다.'}
    ]

}

{'messages': [{'role': 'user', 'content': '지금 몇 시야?'},
  {'role': 'assistant', 'content': '지금은 오후 2시입니다.'}]}

In [12]:
import json

output_file = 'koAlpaca.jsonl'
with open(output_file, 'w', encoding='utf-8') as f:
    for item in dataset['train']:
        instruction = item['instruction']
        output = item['output']
        data = {
            'messages': [
                {'role': 'user', 'content': instruction},
                {'role': 'assistant', 'content': output}
            ]
        }
        json_line = json.dumps(data, ensure_ascii=False)
        f.write(json_line + '\n')

In [13]:
# openai에 업로드
from openai import OpenAI
client = OpenAI()
response = client.files.create(
    file=open("koAlpaca.jsonl", "rb"),
    purpose='fine-tune'
)
print(response.id)  # 파일 업로드 응답 출력

file-3BcYwq8zhx88h1HzCknZE2


In [24]:
# 파인 튜닝
job = client.fine_tuning.jobs.create(
    training_file=response.id,
    model="gpt-4o-mini-2024-07-18",
)
print(job.id)  # 파인 튜닝 작업 ID 출력

ftjob-ynuZd1vns8ojAHNIxiDM4ozM


In [ ]:
# RAG
# 우리회사의 복지제도는?
# LLM은 학습데이터에 없는 최신/특정 정보를 모름

# RAG 해결책  (Tetrieval Augmented Generation) = 검색 + 생성
# 1. 회사 문서에서 관련 정보 검색
# 2. 검색된 정보를 LLM에게 컨텍스트로 제공
# 3. llm이 컨텍스 기반으로 정확한 답변 생성

# [준비 단계]
# 문서들 ->청크분할->벡터변환->벡터DB저장

#[쿼리 단계]
# 질문->벡터변환->유사도검색->상위 k개 선택 -> 컨텍스트 +질문->LLM->답변

In [25]:
%pip install llama-index

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/11.9 MB 8.6 MB/s eta 0:00:02
   ------------- -------------------------- 3.9/11.9 MB 9.9 MB/s eta 0:00:01
   --------------------- ------------------ 6.6/11.9 MB 10.5 MB/s eta 0:00:01
   -------------------------------- ------- 9.7/11.9 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 11.9/11.9 MB 1

  You can safely remove it manually.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
from llama_index.core import Document, VectorStoreIndex

In [ ]:
# 1. 문서 준비
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
documents = [
    Document(text='대한민국의 수도는 서울입니다'),
    Document(text='프랑스의 수도는 파리입니다.'),
]
# 2. 인덱스 생성
# 각 청크를 openai api로 벡터화
# 인메모리방식으로 벡터 스토어에 저장
index = VectorStoreIndex.from_documents(documents)

# 3. 쿼리 엔진 생성
query_engine = index.as_query_engine()

# 4. 쿼리 실행
response = query_engine.query("대한민국의 수도는 어디인가요?")
print(response)  # '서울입니다' 출력

서울


In [31]:
# 청크 : 문서 검색의 최소 단위
# 작을수록 : 정확한 검색, 많은 api 호출
# 클수록 : 넓은 컨텍스트, 적은 api 호출
from llama_index.core import Settings
Settings.chunk_size = 512 # 기본값
# Settings.chunk_overlap = 128 # 기본값
Settings.chunk_overlap = 50 # 청크 간 겹침

In [ ]:
# 한국어 데이터로 Leg 설정
